## Entropy, Mutual Information, KL Divergence

In [1]:
import random
import math
import numpy as np

### The Sims Example

In [2]:
def human_char_behaviour():
    actions = ["Cook a Meal", "Watch TV", "Go to Work", "Read a Book", "Go to Sleep", "Use the Bathroom", "Gardening"]
    return random.choice(actions)

def animal_char_behaviour():
    actions = ["Sleep", "Eat", "Play with Toys"]
    return random.choice(actions)

def simulate_char_behaviour(char_behaviour_function, num_iterations):
    behaviors = [char_behaviour_function() for _ in range(num_iterations)]
    return behaviors

In [3]:
def compute_entropy(behaviours):
    behaviour_counts = {behaviour: behaviours.count(behaviour) for behaviour in set(behaviours)}
    probabilities = [count / len(behaviours) for count in behaviour_counts.values()]
    entropy = -sum(p * math.log2(p) for p in probabilities if p > 0)
    return entropy

In [4]:
num_iterations = 100

human_char_actions = simulate_char_behaviour(human_char_behaviour, num_iterations)
animal_char_actions = simulate_char_behaviour(animal_char_behaviour, num_iterations)

human_char_entropy = compute_entropy(human_char_actions)
animal_char_entropy = compute_entropy(animal_char_actions)

print("Entropy for Human Character Actions:", human_char_entropy) 
print("Entropy for Animal Character Actions:", animal_char_entropy) 

Entropy for Human Character Actions: 2.7645950264217274
Entropy for Animal Character Actions: 1.5653996334852498


In [5]:
def hardworking_human_char_behaviour():
    actions = ["Cook a Meal", "Watch TV", "Go to Work", "Read a Book", "Go to Sleep", "Use the Bathroom", "Gardening"]
    probs = [0.05, 0.05, 0.4, 0.15, 0.15, 0.15, 0.05]
    return random.choices(actions, weights=probs)[0]

num_iterations = 100

human_char_actions = simulate_char_behaviour(human_char_behaviour, num_iterations)
hardworking_human_char_actions = simulate_char_behaviour(hardworking_human_char_behaviour, num_iterations)

human_char_entropy = compute_entropy(human_char_actions)
hardworking_human_char_entropy = compute_entropy(hardworking_human_char_actions)

print("Entropy for (Normal) Human Character Actions:", human_char_entropy) 
print("Entropy for Hardworking Human Character Actions:", hardworking_human_char_entropy) 

Entropy for (Normal) Human Character Actions: 2.7656936811347506
Entropy for Hardworking Human Character Actions: 2.419799105088418


In [6]:
def compute_mutual_info(prob_matrix):
    marginal_x = np.sum(prob_matrix, axis=1)
    marginal_y = np.sum(prob_matrix, axis=0)

    mi = 0.0

    for i in range(prob_matrix.shape[0]):
        for j in range(prob_matrix.shape[1]):
            p_xy = prob_matrix[i, j]
            p_x = marginal_x[i]
            p_y = marginal_y[j]

            # Avoid division by zero
            if p_x > 0 and p_y > 0 and p_xy > 0:
                mi += p_xy * np.log2(p_xy / (p_x * p_y))

    return mi

In [7]:
# joint prob  | Low Energy | High Energy
# ------------|--------------------------          
# Low Hunger  |     0.0    |     0.5
# High Hunger |     0.5    |     0.0

hunger_energy_matrix = np.array([[0, 0.5],
                                 [0.5, 0]])

hunger_energy_mi = compute_mutual_info(hunger_energy_matrix)

print("Mutual information between hunger and energy:", hunger_energy_mi) 

Mutual information between hunger and energy: 1.0


In [8]:
# joint prob  | Bad Cooking | Good Cooking
# ------------|---------------------------          
# Bad Social  |     0.25    |     0.25
# Good Social |     0.25    |     0.25

social_cooking_matrix = np.array([[0.25, 0.25],
                                 [0.25, 0.25]])

social_cooking_mi = compute_mutual_info(social_cooking_matrix)

print("Mutual information between social skill and cooking skill:", social_cooking_mi) 

Mutual information between social skill and cooking skill: 0.0


In [9]:
def compute_kl_divergence(p, q):
    # Avoiding division by zero
    q_nonzero = np.where(q == 0, 1e-10, q)
    kl_divergence = np.sum(p * np.log2(p / q_nonzero))
    return kl_divergence

In [10]:
# True distribution
# -----------------
# joint prob  | Poor | Rich
# ------------|-------------    
# Sad         | 0.30  | 0.10
# Happy       | 0.25  | 0.35

true_happiness_money_matrix = np.array([[0.30, 0.10],
                                        [0.25, 0.35]])

# Estimated distribution (Bad)
# -----------------------------
# joint prob  | Poor | Rich
# ------------|-------------    
# Sad         | 0.5  | 0.0
# Happy       | 0.0  | 0.5

bad_estimated_happiness_money_matrix = np.array([[0.5, 0.0],
                                                 [0.0, 0.5]])

# Estimated distribution (Good)
# -----------------------------
# joint prob  | Poor | Rich
# ------------|-------------    
# Sad         | 0.35  | 0.15
# Happy       | 0.15  | 0.35

good_estimated_happiness_money_matrix = np.array([[0.35, 0.15],
                                                  [0.15, 0.35]])

kl_happiness_money_bad = compute_kl_divergence(true_happiness_money_matrix,
                                               bad_estimated_happiness_money_matrix)
kl_happiness_money_good = compute_kl_divergence(true_happiness_money_matrix,
                                                good_estimated_happiness_money_matrix)

print("KL Divergence (Happiness vs. Money) - Bad Approximator:", kl_happiness_money_bad) 
print("KL Divergence (Happiness vs. Money) - Good Approximator:", kl_happiness_money_good) 

KL Divergence (Happiness vs. Money) - Bad Approximator: 10.393365233876755
KL Divergence (Happiness vs. Money) - Good Approximator: 0.05902742206850159


### More General - Binning method

In [11]:
def entropy(X, bins):
    binned_dist = np.histogram(X, bins)[0]
    probs = binned_dist / np.sum(binned_dist)

    # get rid of bins with zero count
    probs = probs[np.nonzero(probs)]

    entropy = - np.sum(probs * np.log2(probs))

    return entropy

def joint_entropy(X, Y, bins):
    binned_dist = np.histogram2d(X, Y, bins)[0]
    probs = binned_dist / np.sum(binned_dist)
    probs = probs[np.nonzero(probs)]

    joint_entropy = - np.sum(probs * np.log2(probs))

    return joint_entropy

def mutual_info(X, Y, bins):
    H_X = entropy(X, bins)
    H_Y = entropy(Y, bins)
    H_XY = joint_entropy(X, Y, bins)

    MI = H_X + H_Y - H_XY

    return  MI

### Coin Flip Example

In [12]:
X = []
Y = []

# 1000 trials
for i in range(1000):
    x_draw = random.randint(0,1)
    y_draw = random.randint(0,1)
    X.append(x_draw)
    Y.append(y_draw)

In [13]:
print(f'Entropy of X: {entropy(X,2):.3f}')
print(f'Entropy of Y: {entropy(Y,2):.3f}')
print(f'Joint Entropy between X and Y: {joint_entropy(X,Y,2):.3f}')
print(f'Mutual Information between X and Y: {mutual_info(X,Y,2):.3f}')

Entropy of X: 1.000
Entropy of Y: 1.000
Joint Entropy between X and Y: 1.999
Mutual Information between X and Y: 0.000


### Normal Distribution Example

In [14]:
X = []
Y = []

# 1000 trials
for i in range(1000):
    x_draw = np.random.normal(scale=100)
    y_draw = np.random.normal(loc=x_draw)
    X.append(x_draw)
    Y.append(y_draw)

In [15]:
print(f'Entropy of X: {entropy(X,2):.3f}')
print(f'Entropy of Y: {entropy(Y,2):.3f}')
print(f'Joint Entropy between X and Y: {joint_entropy(X,Y,2):.3f}')
print(f'Mutual Information between X and Y: {mutual_info(X,Y,2):.3f}')

Entropy of X: 1.000
Entropy of Y: 1.000
Joint Entropy between X and Y: 1.019
Mutual Information between X and Y: 0.981
